In [56]:
# Import the dependencies
import pandas as pd
import numpy as np

# Import the requests library
import requests

# Import the API key
from config import g_key

# Import dependencies for directions layer
import gmaps
import gmaps.datasets


In [57]:
# Import WeatherPy_vacation csv and save as a new Dataframe
vacation_it_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
vacation_it_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,5,Tiarei,PF,77.00,broken clouds,-17.53,-149.33,Le Rocher de Tahiti
1,6,Sept-Iles,CA,69.80,overcast clouds,50.20,-66.38,Comfort Inn
2,9,Ponta Do Sol,PT,73.99,clear sky,32.67,-17.10,Hotel do Campo
3,10,Wlodawa,PL,63.00,clear sky,51.55,23.55,Ośrodek Wypoczynkowy Ranczo
4,15,Ampanihy,MG,65.70,clear sky,-24.70,44.75,Angora Hotel


In [58]:
# Creating separate DataFrames for four cities within the same continent
flinflon_ca_df = vacation_it_df.loc[(vacation_it_df["City"] == "Flin Flon") & \
                                       (vacation_it_df["Country"] == "CA")]

bonavista_ca_df = vacation_it_df.loc[(vacation_it_df["City"] == "Bonavista") & \
                                       (vacation_it_df["Country"] == "CA")]

vernon_ca_df = vacation_it_df.loc[(vacation_it_df["City"] == "Vernon") & \
                                       (vacation_it_df["Country"] == "CA")]

lakecowi_ca_df = vacation_it_df.loc[(vacation_it_df["City"] == "Lake Cowichan") & \
                                       (vacation_it_df["Country"] == "CA")]

In [59]:
flinflon_ca_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
215,366,Flin Flon,CA,71.6,broken clouds,54.77,-101.86,Victoria Inn Flin Flon


In [60]:
bonavista_ca_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
321,510,Bonavista,CA,66.0,broken clouds,48.65,-53.11,Abbott's Bed & Breakfast


In [61]:
vernon_ca_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
90,169,Vernon,CA,68.0,scattered clouds,50.26,-119.27,Prestige Vernon Lodge and Conference Centre


In [62]:
lakecowi_ca_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
135,248,Lake Cowichan,CA,64.4,broken clouds,48.83,-124.05,Boutique Waterfront Lodge


In [63]:
# Creating latitude and longitude pairs for each city DataFrame as tuples

lat_lng_pairs_df=pd.DataFrame({"Flin Flon": [54.77, -101.86], "Bonavista": [48.65, -53.11], "Vernon": [50.26, -119.27], "Lake Cowichan": [48.83, -124.05]})
lat_lng_pairs_df.to_numpy()

array([[  54.77,   48.65,   50.26,   48.83],
       [-101.86,  -53.11, -119.27, -124.05]])

In [64]:
# Creating direction layer for Bethel to Saint Augustine
fig = gmaps.figure()
fflon2bonavista = gmaps.directions_layer(lat_lng_pairs_df["Flin Flon"], lat_lng_pairs_df["Bonavista"]
                                               ,travel_mode="DRIVING")
bonavista2vernon = gmaps.directions_layer(lat_lng_pairs_df["Bonavista"], lat_lng_pairs_df["Vernon"]
                                               ,travel_mode="DRIVING")
vernon2lakecowi = gmaps.directions_layer(lat_lng_pairs_df["Vernon"], lat_lng_pairs_df["Lake Cowichan"]
                                               ,travel_mode="DRIVING")

fig.add_layer(fflon2bonavista)
fig.add_layer(bonavista2vernon)
fig.add_layer(vernon2lakecowi)

fig

Figure(layout=FigureLayout(height='420px'))

In [78]:
# Merging 4 separate DataFrames for each city into one
df1=pd.concat([flinflon_ca_df, bonavista_ca_df, vernon_ca_df, lakecowi_ca_df], ignore_index=True)
df1

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,366,Flin Flon,CA,71.6,broken clouds,54.77,-101.86,Victoria Inn Flin Flon
1,510,Bonavista,CA,66.0,broken clouds,48.65,-53.11,Abbott's Bed & Breakfast
2,169,Vernon,CA,68.0,scattered clouds,50.26,-119.27,Prestige Vernon Lodge and Conference Centre
3,248,Lake Cowichan,CA,64.4,broken clouds,48.83,-124.05,Boutique Waterfront Lodge


In [70]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [71]:
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in df1.iterrows()]

In [86]:
# Add the vacation spots and marker for each city
locations = df1[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))